In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [27]:
df_data_orig = pd.read_csv('data/iris.csv')
df_data = df_data_orig.copy()
df_data.drop(columns=['Id'], inplace=True)
df_data.head(10)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [28]:
num_classes = len(df_data["Species"].unique())
df_data_enc = pd.get_dummies(df_data, dtype=int)
df_data_enc.head(10)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0
5,5.4,3.9,1.7,0.4,1,0,0
6,4.6,3.4,1.4,0.3,1,0,0
7,5.0,3.4,1.5,0.2,1,0,0
8,4.4,2.9,1.4,0.2,1,0,0
9,4.9,3.1,1.5,0.1,1,0,0


In [29]:
data = torch.tensor(df_data_enc.to_numpy(), dtype=torch.float32)
data.shape

torch.Size([150, 7])

In [30]:
X = data[:, :4]
Y = data[:, 4:]

print(f"{X.shape=}")
print(f"{Y.shape=}")

X.shape=torch.Size([150, 4])
Y.shape=torch.Size([150, 3])


In [31]:
X_norm = (X - X.mean(1, keepdim=True)) / X.std(1, keepdim=True)
X_norm[:10]

tensor([[ 1.1700,  0.4359, -0.5277, -1.0783],
        [ 1.2396,  0.3068, -0.4787, -1.0678],
        [ 1.1765,  0.4255, -0.5257, -1.0763],
        [ 1.1766,  0.3922, -0.4445, -1.1243],
        [ 1.1362,  0.4869, -0.5333, -1.0898],
        [ 1.1431,  0.4707, -0.5155, -1.0982],
        [ 1.1233,  0.5035, -0.5294, -1.0975],
        [ 1.1734,  0.4148, -0.4859, -1.1023],
        [ 1.1932,  0.3703, -0.4526, -1.1109],
        [ 1.2084,  0.3384, -0.4350, -1.1117]])

In [32]:
n1 = int(0.8 * len(X))
n2 = n1 + int(0.1 * len(X))

X_train = X_norm[:n1]
Y_train = Y[:n1]
X_val = X_norm[n1:n2]
Y_val = Y[n2:]
X_test = X_norm[n1:n2]
Y_test = Y[n2:]

X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_val = X_val.to(device)
Y_val = Y_val.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)

print(f"{X_train.shape=}")
print(f"{Y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{Y_val.shape=}")
print(f"{X_test.shape=}")
print(f"{Y_test.shape=}")

X_train.shape=torch.Size([120, 4])
Y_train.shape=torch.Size([120, 3])
X_val.shape=torch.Size([15, 4])
Y_val.shape=torch.Size([15, 3])
X_test.shape=torch.Size([15, 4])
Y_test.shape=torch.Size([15, 3])


In [33]:
class Model(nn.Module):

    def __init__(self, n_hidden) -> None:
        super().__init__()
        self.l1 = nn.Linear(4, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_hidden)
        self.l3 = nn.Linear(n_hidden, n_hidden)
        self.l4 = nn.Linear(n_hidden, 3)

    def forward(self, x):
        x = F.tanh(self.l1(x))
        x = F.tanh(self.l2(x))
        x = F.tanh(self.l3(x))
        return F.softmax(self.l4(x), 1)


In [34]:
model = Model(16)
m = model.to(device)
m

Model(
  (l1): Linear(in_features=4, out_features=16, bias=True)
  (l2): Linear(in_features=16, out_features=16, bias=True)
  (l3): Linear(in_features=16, out_features=16, bias=True)
  (l4): Linear(in_features=16, out_features=3, bias=True)
)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [36]:
loss = None

for epoch in range(10000):
    optimizer.zero_grad()
    outputs = m(X_train)
    loss = criterion(outputs, Y_train)
    loss.backward()
    optimizer.step()

print(loss)

tensor(0.5691, device='cuda:0', grad_fn=<DivBackward1>)
